<a href="https://colab.research.google.com/github/joshi-nitant/083_Nitant_Joshi/blob/master/Lab5/083_1_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch

A tensor is a number, vector, matrix or any n-dimensional array.

## Problem Statement

We'll create a model that predicts crop yeilds for apples (*target variable*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in different regions. 

Here's the training data:

>Temp | Rain | Humidity | Prediction
>--- | --- | --- | ---
> 73 | 67 | 43 | 56
> 91 | 88 | 64 | 81
> 87 | 134 | 58 | 119
> 102 | 43 | 37 | 22
> 69 | 96 | 70 | 103

In a **linear regression** model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
```

It means that the yield of apples is a linear or planar function of the temperature, rainfall & humidity.



**Our objective**: Find a suitable set of *weights* and *biases* using the training data, to make accurate predictions.

## Training Data
The training data can be represented using 2 matrices (inputs and targets), each with one row per observation and one column for variable.

In [2]:

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Target (apples)
targets = np.array([[56], 
                    [81], 
                    [119], 
                    [22], 
                    [103]], dtype='float32')

Before we build a model, we need to convert inputs and targets to PyTorch tensors.



In [3]:
# Convert inputs and targets to tensors
tensor_inputs = torch.tensor(inputs)
print("tensor inputs:\n",tensor_inputs)

tensor_targets = torch.tensor(targets)
print("\ntensor targets:\n",tensor_targets)

tensor inputs:
 tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

tensor targets:
 tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


## Linear Regression Model (from scratch)

The *weights* and *biases* can also be represented as matrices, initialized with random values. The first row of `w` and the first element of `b` are use to predict the first target variable i.e. yield for apples, and similarly the second for oranges.

In [4]:
# Weights and biases
weights = torch.randn(1, 3, requires_grad=True)
biases = torch.randn(1, requires_grad=True)
weights_transpose = torch.transpose(weights, 0, 1)

print("Weights:\n",weights)
print("\nBiases:\n",biases)
print("\nweights_transpose:\n",weights_transpose)

Weights:
 tensor([[ 1.5939,  0.1444, -1.5983]], requires_grad=True)

Biases:
 tensor([0.0007], requires_grad=True)

weights_transpose:
 tensor([[ 1.5939],
        [ 0.1444],
        [-1.5983]], grad_fn=<TransposeBackward0>)


The *model* is simply a function that performs a matrix multiplication of the input `x` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

$$
\hspace{2.5cm} X \hspace{1.1cm} \times \hspace{1.2cm} W^T \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\times
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$

In [5]:
# Define the model
def fit(X, W_t, b): 
    return (X @ W_t) + b

The matrix obtained by passing the input data to the model is a set of predictions for the target variables.

In [6]:

# Generate predictions
prediction = fit(tensor_inputs, weights_transpose, biases)
print(prediction)

tensor([[ 57.3045],
        [ 55.4634],
        [ 65.3223],
        [109.6498],
        [ 11.9641]], grad_fn=<AddBackward0>)


In [7]:
# Compare with targets
print(tensor_targets)


tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


Because we've started with random weights and biases, the model does not perform a good job of predicting the target varaibles.

## Loss Function

We can compare the predictions with the actual targets, using the following method: 
* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [8]:
# MSE loss
# help(torch.numel)
def mse_loss(pred, target):
    temp = pred - target
    return torch.sum(temp * temp) / temp.numel()

In [10]:
# Compute loss
loss = mse_loss(prediction, tensor_targets)
print("Loss: ", loss)


Loss:  tensor(3901.0281, grad_fn=<DivBackward0>)


The resulting number is called the **loss**, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model. 

## Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the `loss` w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

More on autograd:  https://pytorch.org/docs/stable/autograd.html#module-torch.autograd

In [11]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors.

In [12]:
# Gradients for weights
print(weights, end="\n-------------\n")
print(weights.grad)

tensor([[ 1.5939,  0.1444, -1.5983]], requires_grad=True)
-------------
tensor([[-1695.9038, -5729.2554, -3128.4104]])


In [13]:
# Gradients for bias
print(biases, end="\n-------------\n")
print(biases.grad)

tensor([0.0007], requires_grad=True)
-------------
tensor([-32.5184])


A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases. 

* If a gradient element is **postive**, 
    * **increasing** the element's value slightly will **increase** the loss.
    * **decreasing** the element's value slightly will **decrease** the loss.




* If a gradient element is **negative**,
    * **increasing** the element's value slightly will **decrease** the loss.
    * **decreasing** the element's value slightly will **increase** the loss.
    


The increase or decrease is proportional to the value of the gradient.

Finally, we'll reset the gradients to zero before moving forward, because PyTorch accumulates gradients.

In [14]:
# help(biases.grad.zero_)
weights.grad.zero_()
biases.grad.zero_()
print(weights.grad,end="\n-------\n")
print(biases.grad)

tensor([[0., 0., 0.]])
-------
tensor([0.])


## Adjust weights and biases using gradient descent
​
We'll reduce the loss and improve our model using the gradient descent algorithm, which has the following steps:
​
1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [15]:
# Generate predictions
prediction = fit(tensor_inputs, weights.t(), biases)
print(prediction)

tensor([[ 57.3045],
        [ 55.4634],
        [ 65.3223],
        [109.6498],
        [ 11.9641]], grad_fn=<AddBackward0>)


In [16]:
# Calculate the loss
loss = mse_loss(prediction, tensor_targets)
print(loss)

tensor(3901.0281, grad_fn=<DivBackward0>)


In [17]:
# Compute gradients
loss.backward()

In [18]:

# Adjust weights & reset gradients
with torch.no_grad():
    weights -= weights.grad * 1e-5
    biases -= biases.grad * 1e-5
    weights.grad.zero_()
    biases.grad.zero_()

In [19]:

print(weights)

tensor([[ 1.6108,  0.2017, -1.5670]], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [20]:

# Calculate loss
prediction = fit(tensor_inputs, weights.t(), biases)
loss = mse_loss(prediction, tensor_targets)
print(loss)

tensor(3514.6265, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.

In [21]:
# Train for 100 epochs
for i in range(100):
    prediction = fit(tensor_inputs, weights.t(), biases)
    loss = mse_loss(prediction, tensor_targets)
    loss.backward()
    with torch.no_grad():
        weights -= weights.grad * 1e-5
        biases -= biases.grad * 1e-5
        weights.grad.zero_()
        biases.grad.zero_()

In [22]:
# Calculate loss
prediction = fit(tensor_inputs, weights.t(), biases)
loss = mse_loss(prediction, tensor_targets)
print(loss)

tensor(547.5480, grad_fn=<DivBackward0>)


In [23]:
# Print predictions
print(prediction)

tensor([[ 63.3302],
        [ 73.0416],
        [129.6265],
        [ 55.5753],
        [ 65.8456]], grad_fn=<AddBackward0>)


In [24]:
# Print targets
print(tensor_targets)

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])
